# SPAM Dectection using Support Vector Machines

## Setup

<p>Add intelex accelerator to speed up SVM algorithm</p>

In [ ]:
#!pip install scikit-learn-intelex

In [ ]:
#import sklearnx
#patch_sklearn()

### Imports

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

### File Structure Constants

In [ ]:
FOLDER_DS1 = "Datasets/glasses-and-coverings/"
FOLDER_DS2 = "Datasets/glasses-and-noglasses/"
FOLDER = "Datasets/combined_datasets/"
# folders within glasses-and-covering
covering = "covering"
glasses_ds1 = "glasses_ds1"
plain = "plain"
sunglasses = "sunglasses"
sunglasses_imagenet = "sunglasses-imagenet"
# folder within glasses-and-noglasses
glasses_ds2 = "glasses_ds2"
no_glasses = "no_glasses"
#Combined dataset
glasses ='glasses'

#### Check paths are correct

In [ ]:
pth = os.path.join(FOLDER_DS2,glasses_ds2)
print(os.listdir(pth))

## Data Exploration

<p>Inner folder names</p>

In [ ]:
# Folders we are interested in
folder_ds1 = [glasses_ds1,plain,sunglasses,sunglasses_imagenet]
folder_ds2 = [glasses_ds2, no_glasses]
folder_combined = [glasses]
pth = os.path.join(FOLDER_DS2,folder_ds2[0])
print(os.listdir(pth))

<p>Create function to explore data</p>

In [ ]:
from collections import defaultdict

#Create function to explore datasets
def getfolderinfo(FOLDER_PATH,folders):
    # Define a default dict to store folder properties
    folder_info = defaultdict(dict)
    for folder in folders:
        path = os.path.join(FOLDER_PATH, folder)
        length = len(os.listdir(path))
        folder_info[folder]['length'] = length
        suffix_count = defaultdict(int)
        not_image = 0
        # Check files in folder
        for img in os.listdir(path):
            # Check suffix
            suffix = img.split('.')[-1]
            # Define these suffix as valid image files
            if suffix.lower() in ['jpg', 'jpeg', 'png', 'gif']:
                suffix_count[suffix] += 1
            else:
                not_image += 1

            # Save folder info            
            folder_info[folder]['suffix_frequency'] = suffix_count
            folder_info[folder]['not_image'] = not_image

    return folder_info

   

<p>Execute function on dataset folders</p>

In [ ]:
folder_info_ds1 = getfolderinfo(FOLDER_DS1,folder_ds1)
folder_info_ds2 = getfolderinfo(FOLDER_DS2, folder_ds2)

### Glasses-and-coverings dataset

<p>Visualise results.<br>These results shows that the dataset is balanced in terms of numbers of images so there is not imbalance in the data which not cause bias.</p>

In [ ]:
keys = list(folder_info_ds1.keys())
num_images = [folder_info_ds1[folder]['length'] for folder in keys]

plt.figure(figsize=(10, 6))
plt.bar(keys, num_images, color='skyblue')
plt.xlabel('Folders')
plt.ylabel('Number of Images')
plt.title('Number of Images in each Folder of Dataset 1')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

<p> All files in folders are images with most files being jpg in this dataset with small exception. This should cause no problems going forward.</p>

In [ ]:
not_image_count = 0

suffix_counts = defaultdict(int)
not_image_count = 0

for folder_data in folder_info_ds1.values():
    suffix_info = folder_data['suffix_frequency']
    for suffix, count in suffix_info.items():
        suffix_counts[suffix] += count
    not_image_count += folder_data['not_image']

suffix_labels = list(suffix_counts.keys())
suffix_freqs = list(suffix_counts.values())

plt.figure(figsize=(10, 6))
plt.bar(suffix_labels, suffix_freqs, color='skyblue')
plt.bar('Not Image', not_image_count, color='lightgreen')
plt.xlabel('Suffix or Not Image')
plt.ylabel('Count')
plt.title('Frequency of Suffixes and Non-Image Files of Dataset 1')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Glasses-and-noglasses dataset

<p>Visualise results<br>
Dataset is fairly balanced with a slight bias towards images with glasses with the difference about 16%. This may cause a slight bias but these datasets will be combined so the variance can be rechecked then.</p>

In [ ]:
folders = list(folder_info_ds2.keys())
num_images = [folder_info_ds2[folder]['length'] for folder in folders]

plt.figure(figsize=(10, 6))
plt.bar(folders, num_images, color='skyblue')
plt.xlabel('Folders')
plt.ylabel('Number of Images')
plt.title('Number of Images in each Folder of Dataset 2')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

<p> All files in folders are images with all files being jpg in this dataset. This should cause no problems going forward.</p>

In [ ]:
not_image_count = 0

suffix_counts = defaultdict(int)
not_image_count = 0

for folder_data in folder_info_ds2.values():
    suffix_info = folder_data['suffix_frequency']
    for suffix, count in suffix_info.items():
        suffix_counts[suffix] += count
    not_image_count += folder_data['not_image']

suffix_labels = list(suffix_counts.keys())
suffix_freqs = list(suffix_counts.values())

plt.figure(figsize=(10, 6))
plt.bar(suffix_labels, suffix_freqs, color='skyblue')
plt.bar('Not Image', not_image_count, color='lightgreen')
plt.xlabel('Suffix or Not Image')
plt.ylabel('Count')
plt.title('Frequency of Suffixes and Non-Image Files of Dataset 2')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Combined dataset

In [ ]:
folders = [plain, glasses, sunglasses]

<p>Visualise results<br>
Combining these datasets leads to a large imbalance in numbers of images with sunglasses. This has the potential to bias the model against the sunglasses class. To rectify, when creating training data, we could take all images from sunglasses and equal images from glasses and plain.</p>

In [ ]:
combined_size = []
for folder in folders:
    path = os.path.join(FOLDER, folder)
    length = len(os.listdir(path))
    combined_size.append(length)

plt.figure(figsize=(10, 6))
plt.bar(folders, combined_size, color='skyblue')
plt.xlabel('Folders')
plt.ylabel('Number of Images')
plt.title('Number of Images in each Folder of Combined dataset')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

<p>Sample Images</p>

In [ ]:
lists = os.listdir(os.path.join(FOLDER,'glasses'))
pth = os.path.join(FOLDER,'glasses', lists[0])
img_array = cv2.imread(pth, cv2.IMREAD_GRAYSCALE)
backtorgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
plt.imshow(img_array, cmap="gray")

In [ ]:
fig, ax = plt.subplots(3, 5, figsize=(8, 6))
for i, axi in enumerate(ax.flat):
    lists = os.listdir(os.path.join(FOLDER,'glasses'))
    pth = os.path.join(FOLDER,'glasses', lists[i])
    img_array = cv2.imread(pth, cv2.IMREAD_GRAYSCALE)
    backtorgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
    axi.imshow(img_array, cmap="gray")
    axi.set(xticks=[], yticks=[],
            xlabel='glasses')

## Data Preprocessing

<p>Define labels</p>

In [ ]:
labels = [ glasses, plain, sunglasses]

<p>Create function to process images to training data<p>

In [ ]:
##Reading all the images and converting to array for data and labels

def create_training_data(FOLDER, folders):
    training_data = []
    for folder in folders:
        path = os.path.join(FOLDER, folder)
        class_num = folders.index(folder) #0,1
        for img in os.listdir(path)[:len(os.listdir(path))]:
            try:
                img_array = cv2.imread(os.path.join(path + "\\" + img), cv2.IMREAD_GRAYSCALE)
                backtorgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
                new_array = cv2.resize(backtorgb, (256, 256))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass 
    return training_data


<p>Combined dataset training data</p>

In [ ]:
combined_dataset = create_training_data( FOLDER, folders)

In [ ]:
combined_dataset[0]

## Feature Vector and Target Variable

In [ ]:
X = []
y = []

for features,label in combined_dataset:
    X.append(features)
    y.append(label)
    
X = np.array(X)
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)

In [ ]:
X_train.shape

<p>Flatten image for each entry to be a single column with however many columns are needed (-1)

In [ ]:
# Flatten images
X_train = X_train.reshape(X_train.shape[0],-1)
X_train.shape

In [ ]:
# Flatten images
X_test = X_test.reshape(X_test.shape[0],-1)
X_test.shape

## Model Creation

<p> First model, simple Support Vector Machine with kernel set to rbf. Class_weight set to balanced.<br>
Kernel: RBF Radial Balance Function-effective for non-linear relationships in data.<br>
Class_weight: Can be unsed to balance imbalanced class distributions. eg. sunglasses.</p?>
 <p>Result: Dataset was too large for SVM. SVMs have poor performance on large dataset due to the exponential calculations as the dataset grows.<p>

In [ ]:
## Dataset too large for model

#from sklearn.svm import SVC
#model = SVC(kernel='rbf', class_weight='balanced')
#model.fit(X_train,y_train)


<p> Next model, use Principal Component Analysis to see if this can reduce the data the SVM must calculate a fit for. This model created a fit in around 120 seconds compared to the previous which ran for 20 minutes before stopping execution. This shows that using PCA is essential when utilizing the standard SVM algorithm on large datasets, <p>

In [ ]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

#Set components to 150
pca = PCA(n_components=150, whiten=True,
          svd_solver='randomized', random_state=42)
svc = SVC(kernel='rbf', class_weight='balanced')
model = make_pipeline(pca, svc)

In [ ]:
#model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

<p> We can see from these results that the model performs well on glasses and no glasses but poorly on sunglasses. So the imbalance in the dataset does indeed seemed to cause bias in our model.</p>

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=labels))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat.T, square=True, annot=True, fmt='d',
            cbar=False, cmap='Blues',
            xticklabels=labels,
            yticklabels=labels)
plt.xlabel('true label')
plt.ylabel('predicted label')

<p>Model 3: In this model we will attempt to improved accuracy on sunglasses using the class_weight property of the SVM model</p>

<p>First we need to calculation weights for each class</p>

In [ ]:
combined_sum = sum (combined_size) 
#float(combined_sum)
combined_sum

In [ ]:
entries = len(combined_size)
entries

In [ ]:
class_weights=[]
for i, size in enumerate(combined_size):
    class_weights.append( str(combined_sum / (entries * size)))
class_weights

In [ ]:
#Apparently, it must be a  dict
class_weights_dict = {0: class_weights[0], 1: class_weights[1], 2: class_weights[2]} 
class_weights_dict[0]

<p>Then add the class weights to model</p>

In [ ]:
pca = PCA(n_components=150, whiten=True,
          svd_solver='randomized', random_state=42)
svc = SVC(kernel='rbf', class_weight=class_weights_dict)
model = make_pipeline(pca, svc)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)
from  sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=labels))


In [ ]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X)
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)

In [ ]:
# Flatten images
X_test = X_test.reshape(X_test.shape[0],-1)
X_test.shape

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat.T, square=True, annot=True, fmt='d',
            cbar=False, cmap='Blues',
            xticklabels=labels,
            yticklabels=labels)
plt.xlabel('true label')
plt.ylabel('predicted label');

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid)

%time grid.fit(X_train, y_train)
print(grid.best_params_)

In [ ]:
model = grid.best_estimator_
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=labels))

In [ ]:
mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat.T, square=True, annot=True, fmt='d',
            cbar=False, cmap='Blues',
            xticklabels=labels,
            yticklabels=labels)
plt.xlabel('true label')
plt.ylabel('predicted label')

<p>With no PCA, lower accuracy</p>

In [ ]:
svc = SVC(kernel='sigmoid', class_weight='balanced')
svc.fit(X_train,y_train)
y_pred = svc.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=labels))

In [ ]:
model = grid.best_estimator_
y_pred = model.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=labels))